# Task3-1

In [ ]:
!pip install kgtk

In [ ]:
!pip install ampligraph

In [3]:
from google.colab import files

In [ ]:
!echo "deb http://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
!apt-key adv --keyserver keys.openpgp.org --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-cairo python3-matplotlib
!apt-get install libcairo2-dev

In [5]:
import numpy as np
import pandas as pd
import requests
import ampligraph
from ampligraph.datasets import load_from_csv

In [6]:
url = 'https://ampligraph.s3-eu-west-1.amazonaws.com/datasets/GoT.csv'
open('GoT.csv', 'wb').write(requests.get(url).content)
X = load_from_csv('.', 'GoT.csv', sep=',')

In [7]:
X

array([['Smithyton', 'SEAT_OF', 'House Shermer of Smithyton'],
       ['House Mormont of Bear Island', 'LED_BY', 'Maege Mormont'],
       ['Margaery Tyrell', 'SPOUSE', 'Joffrey Baratheon'],
       ...,
       ['Extinct', 'SEAT_OF', 'House Hook'],
       ['House Marsh', 'SWORN_TO', 'House Bolton of the Dreadfort'],
       ['House Marbrand of Ashemark', 'IN_REGION', 'The Westerlands']],
      dtype=object)

In [8]:
import io
import os
import sys
import pandas as pd
import json
from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher
 
df = pd.DataFrame(X)
df = df.rename({0 : "node1", 1 : "label", 2 : "node2"}, axis = 1)
df.index = df.node1
df = df.drop("node1", axis = 1)
df.reindex()
 
data_dir = '.'
df.to_csv(data_dir + "/data.tsv", sep='\t')
extra_files_config = {"data": "data.tsv"}
open(data_dir + '/extra_files.json', 'w').write(json.dumps(extra_files_config))
 
input_path = data_dir
output_path = data_dir
input_files_url = data_dir
project_name = 'Evan_project'
 
files = ["data"]
ck = ConfigureKGTK(files, input_files_url=input_files_url)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name,
                  json_config_file = data_dir + '/extra_files.json',)
 
ck.print_env_variables()
ck.load_files_into_cache()

User home: /root
Current dir: /content
KGTK dir: /
Use-cases dir: //use-cases
EXAMPLES_DIR: //examples
STORE: ./Evan_project/temp.Evan_project/wikidata.sqlite3.db
KGTK_OPTION_DEBUG: false
kypher: kgtk query --graph-cache ./Evan_project/temp.Evan_project/wikidata.sqlite3.db
OUT: ./Evan_project
TEMP: ./Evan_project/temp.Evan_project
USE_CASES_DIR: //use-cases
KGTK_GRAPH_CACHE: ./Evan_project/temp.Evan_project/wikidata.sqlite3.db
KGTK_LABEL_FILE: ./labels.en.tsv.gz
GRAPH: .
kgtk: kgtk
data: ./data.tsv
data: ./data.tsv
kgtk query --graph-cache ./Evan_project/temp.Evan_project/wikidata.sqlite3.db -i "./data.tsv" --as data  -i "./data.tsv" --as data  --limit 3
node1	label	node2
Smithyton	SEAT_OF	House Shermer of Smithyton
House Mormont of Bear Island	LED_BY	Maege Mormont
Margaery Tyrell	SPOUSE	Joffrey Baratheon


In [9]:
%%time
kgtk("""
    graph-statistics -i "$data" -o $OUT/metadata.pagerank.directed.tsv.gz 
    --compute-pagerank True 
    --compute-hits False 
    --page-rank-property Pdirected_pagerank 
    --vertex-in-degree-property Pindegree
    --vertex-out-degree-property Poutdegree
    --output-degrees True 
    --output-pagerank True 
    --output-hits False \
    --output-statistics-only 
    --undirected False 
    --log-file $TEMP/metadata.pagerank.directed.summary.txt
""")

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/all.py:39: RuntimeWarning: Error importing draw module, proceeding nevertheless: No module named 'cairo._cairo'
  warnings.warn(msg, RuntimeWarning)

CPU times: user 43.8 ms, sys: 26.8 ms, total: 70.6 ms
Wall time: 5.13 s


In [27]:
kgtk(""" 
    head -i $OUT/metadata.pagerank.directed.tsv.gz
""")

,node1,label,node2,id
0,Smithyton,Pindegree,0.000000,Smithyton-Pindegree-0
1,Smithyton,Poutdegree,1.000000,Smithyton-Poutdegree-1
2,Smithyton,Pdirected_pagerank,0.000142,Smithyton-Pdirected_pagerank-2
3,House Shermer of Smithyton,Pindegree,1.000000,House Shermer of Smithyton-Pindegree-3
4,House Shermer of Smithyton,Poutdegree,2.000000,House Shermer of Smithyton-Poutdegree-4
5,House Shermer of Smithyton,Pdirected_pagerank,0.000262,House Shermer of Smithyton-Pdirected_pagerank-5
6,House Mormont of Bear Island,Pindegree,10.000000,House Mormont of Bear Island-Pindegree-6
7,House Mormont of Bear Island,Poutdegree,3.000000,House Mormont of Bear Island-Poutdegree-7
8,House Mormont of Bear Island,Pdirected_pagerank,0.001431,House Mormont of Bear Island-Pdirected_pagerank-8
9,Maege Mormont,Pindegree,1.000000,Maege Mormont-Pindegree-9


In [33]:
# Show top-20 nodes with the highest pagerank
pagerank_top20 = kgtk(""" 
    query -i data -i $OUT/metadata.pagerank.directed.tsv.gz \
      --match '
        data: (node)-[]->(),
        pagerank: (node)-[:Pdirected_pagerank]->(pagerank) '
      --return 'distinct node, pagerank'
      --order-by 'pagerank desc'
      --limit 20
""")
pagerank_top20

,node1,node2
0,House Baratheon of King's Landing,0.044739
1,House Lannister of Casterly Rock,0.021143
2,House Tyrell of Highgarden,0.018228
3,House Baratheon of Storm's End,0.016333
4,House Baratheon of Dragonstone,0.015298
5,House Frey of the Crossing,0.014261
6,House Nymeros Martell of Sunspear,0.014188
7,House Stark of Winterfell,0.014090
8,House Targaryen of King's Landing,0.012127
9,House Baelish of Harrenhal,0.012000


In [35]:
# Show top-20 nodes with the highest in-degree
indegree_top20 = kgtk(""" 
    query -i data -i $OUT/metadata.pagerank.directed.tsv.gz \
      --match '
        data: (node)-[]->(),
        pagerank: (node)-[:Pindegree]->(indegree) '
      --return 'distinct node, indegree'
      --order-by 'cast(indegree, float) desc'
      --limit 20
""")
indegree_top20

,node1,node2
0,House Lannister of Casterly Rock,133
1,House Frey of the Crossing,119
2,House Tyrell of Highgarden,102
3,House Stark of Winterfell,100
4,House Targaryen of King's Landing,94
5,House Greyjoy of Pyke,67
6,House Baratheon of King's Landing,56
7,House Baratheon of Storm's End,56
8,House Nymeros Martell of Sunspear,54
9,House Arryn of the Eyrie,52


In [41]:
num = 0
lst1 = list(pagerank_top20.node1)
lst2 = list(indegree_top20.node1)
for i in lst1:
  if i in lst2:
    num += 1
print('The number of overlaps:', num)

The number of overlaps: 15


# Task 3-2. Extended Path

In [26]:
%%time
kgtk("""
    reachable-nodes -i $data
        --root 'House Lannister of Casterly Rock'
        --prop 'ALLIED_WITH'
        --inverted-prop 'ALLIED_WITH'
""")

CPU times: user 12.3 ms, sys: 18.5 ms, total: 30.8 ms
Wall time: 1.12 s


,node1,label,node2
0,House Lannister of Casterly Rock,reachable,Weese
1,House Lannister of Casterly Rock,reachable,Stafford Lannister
2,House Lannister of Casterly Rock,reachable,Amory Lorch
3,House Lannister of Casterly Rock,reachable,Tion Lannister
4,House Lannister of Casterly Rock,reachable,Tygett Lannister
...,...,...,...
73,House Lannister of Casterly Rock,reachable,Tytos Lannister
74,House Lannister of Casterly Rock,reachable,Myranda Lefford
75,House Lannister of Casterly Rock,reachable,Cadwyn
76,House Lannister of Casterly Rock,reachable,Tybolt Lannister
